# Study 1

We create a Spark session. We select 7 cores for execute the codesnippet in parallel.

In [10]:
 spark = SparkSession.builder\
      .master("local[7]")\
      .appName("Study2")\
      .getOrCreate()

We start to measure the time. Then, we first transform the variable *tpep_pickup_datetime* into *datetime* format. Then, we extract the hour and create a new column. Finally this new variable is transformed into categorical with 4 levels. 

In [ ]:
start = time.time()
data_spark = spark.read.option("header","true").csv("DATA/*.csv")
print((data_spark.count(), len(data_spark.columns)))


data_sp = data_spark.withColumn("tpep_pickup_timestap",to_timestamp("tpep_pickup_datetime")) 
data_sp = data_sp.withColumn("tpep_pickup_hour", hour(to_timestamp("tpep_pickup_timestap","yyyy-MM-dd HH:mm:ss")))
data_sp = data_sp.withColumn("tpep_pickup_timezone", when((col("tpep_pickup_hour") >= 0) & (col("tpep_pickup_hour") < 6),"N")
                                 .when((col("tpep_pickup_hour") >= 6) & (col("tpep_pickup_hour") < 12),"M")
                                 .when((col("tpep_pickup_hour") >= 12) & (col("tpep_pickup_hour") < 18),"A")
                                 .when((col("tpep_pickup_hour") >= 18) & (col("tpep_pickup_hour") <= 24),"E")
                                 .when(col("tpep_pickup_hour").isNull() ,""))
                            
data_sp.groupBy('tpep_pickup_timezone').count().show()
data_sp = data_sp.withColumn("tpep_pickup_timezone", data_sp["tpep_pickup_timezone"].cast(IntegerType()))
spark.stop()
end = time.time()
print("Time elapsed:", end-start, "seconds")